### PSO optimizes a problem by:
### 1. having a population of candidate solutions, here dubbed particles
### 2. moving these particles around in the search-space according to simple mathematical formulae. 

##### The movements of the particles are guided by the best found positions in the search-space which are updated as better positions are found by the particles.

In [70]:
import operator
import random
import numpy

from inspect import getsource
#from dill.source import getsource

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

In [48]:
t1=linspace(-50,50,100)
sig1=sin(t1/2)+np.random.normal(scale=0.1,size=len(t1))
sig2=sin(t1/2)+np.random.normal(scale=0.1,size=len(t1))


f0=interp1d(t1,sig1,kind="cubic",bounds_error=False,fill_value=-10000)
g=interp1d(t1,sig2,kind="cubic",bounds_error=False,fill_value=10000)
#true value that  I would like to estimate
A=2
B=0.8
C=0

Ig=arange(t1.min(),t1.max(),1)
If=(Ig-A)/B
I_min=max(If.min(),Ig.min())
I_max=min(If.max(),Ig.max())


J=linspace(min(If.min(),Ig.min()),max(If.max(),Ig.max()))
I=linspace(I_min,I_max,1000)  

### Functions to maximize (or minimize)

In [49]:
def s(t):
    return A+B*t+C*t*t

def inv_s(t):
    return (t-B)/A

def f(t):
    return f0(s(t))

def hat_s(t,a,b):
    return t*b+a

def cost(x,T=I):
    a=x[0]
    b=x[1]
    return norm(g(b*T+a)-f(T))/len(T),

### PSO particles are essentially described as a positions in a search-space of D dimensions

##### Each particle also has a vector representing the speed of the particle in each dimension. Finally, each particle keeps a reference to the best state in which it has been so far.

#### 1. We want to maximise the value of the fitness of our particles -> "FitnessMax"
#### 2. The second object "Particle" we create represent our particle. We defined it as a list to which we add five attributes. The first attribute is the fitness of the particle, the second is the speed of the particle which is also goind to be a list, the third and fourth are the limit of the speed value, and the fifth attribute will be a reference to a copy of the best state the particle has been so far.

In [36]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

### PSO original algorithm use three operators : initializer, updater and evaluator.


#### The initialization "generate" consist in generating a random position and a random speed for a particle.

In [37]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

### The function updateParticle() :
#### 1. First computes the speed
#### 2. Limits the speed values between smin and smax
#### 3. Finally computes the new particle position.

In [38]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))


### Evaluation

In [39]:
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", cost)

In [46]:
def setParams():
    
    print pop

In [47]:
setParams()

[[1.4432189486722136, -4.491471511744211], [-5.168306710094598, 0.7910756420513891], [-2.3482861214654402, 0.7612051262119408], [-4.128916521830329, 3.9533538621952964], [-5.236390392948599, 0.8237296460232608]]


### Create new population and Optimize

In [62]:
def optimize(numPoints,numIter):
   
    pop = toolbox.population(n=numPoints)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = numIter
    best = None
    
    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            print part.fitness.values
#             if not part.best or part.best.fitness < part.fitness:
#                 part.best = creator.Particle(part)
#                 part.best.fitness.values = part.fitness.values
#             if not best or best.fitness < part.fitness:
#                 best = creator.Particle(part)
#                 best.fitness.values = part.fitness.values
#         for part in pop:
#             toolbox.update(part, best)

#         # Gather all the fitnesses in one list and print the stats
#         logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
#         print(logbook.stream)
#         print "  Best so far: %s - %s" % (best, best.fitness)

#     return pop, logbook, best


In [63]:
optimize(2,1) #pop,logbook,best= optimize()
# print "best=",best,"A,B=",(A,B)

(248.79608691668255,)
(194.67723266822469,)


In [64]:
print pop[0]
cost(pop)

[-0.5366073372687192, 1.3932321643488068]


ValueError: operands could not be broadcast together with shapes (2,) (1000,) 

In [73]:
print getsource(evaluate)

NameError: name 'evaluate' is not defined